In [37]:
import numpy as np
import os, sys
import matplotlib.pyplot as plt

from PyGEECSPlotter.scan_data_analysis import ScanDataAnalyzer
from PyGEECSPlotter.navigation_utils import get_top_dir, generate_sfilename_list_from_scans_dir

import ipywidgets as widgets
import datetime

import pandas as pd

In [38]:
# Can use search_replace to change column names to more useful names
# Can use column_math to quickly turn column values into more useful quantites (e.g. stage positions)

column_math_filename = '/Users/alexpicksley/Documents/GitHub/bella_analysis/user_data/2024-02-29_columnmath_testguiding.txt'
search_replace_filename = '/Users/alexpicksley/Documents/GitHub/bella_analysis/user_data/2024-02-29_searchreplace.txt'



In [3]:
experiment_dir = r'N:/data/'
# experiment_dir = r'D:/Server/Data/PW/'
style = {}

start_date_widget = widgets.DatePicker(description='Start Date', value = datetime.date.today(), style=style, disabled=False)
end_date_widget = widgets.DatePicker(description='End Date', value = datetime.date.today(), style=style, disabled=False)
dates_box = widgets.HBox([start_date_widget, end_date_widget])
dates_box

In [39]:
start_date = start_date_widget.value
end_date = end_date_widget.value

date_list = pd.date_range(start_date, end_date, freq='D')
year_month_day_list = [(d.year, d.month, d.day) for d in date_list]

year_month_day_list

[(2025, 7, 27),
 (2025, 7, 28),
 (2025, 7, 29),
 (2025, 7, 30),
 (2025, 7, 31),
 (2025, 8, 1),
 (2025, 8, 2),
 (2025, 8, 3),
 (2025, 8, 4),
 (2025, 8, 5),
 (2025, 8, 6),
 (2025, 8, 7),
 (2025, 8, 8),
 (2025, 8, 9),
 (2025, 8, 10),
 (2025, 8, 11),
 (2025, 8, 12),
 (2025, 8, 13),
 (2025, 8, 14)]

In [40]:

sfilename_list = []
for i in range(len(year_month_day_list)):
    year, month, day = year_month_day_list[i]
    top_dir = get_top_dir(experiment_dir, year, month, day)
    
    tmp_list = generate_sfilename_list_from_scans_dir(top_dir)
    for sfilename in tmp_list:
        if os.path.exists(sfilename):
            sfilename_list.append( sfilename )

In [41]:
# Load the Scalar Data for that scan in, and filter any data if desired

all_parameters_list  = []
for sfilename in sfilename_list:
    scan_data = ScanDataAnalyzer(sfilename=sfilename)
    
    scan_data.load_scan_data(
        # search_replace_filename=search_replace_filename, 
        # column_math_filename=column_math_filename,
    )

    for col_name in scan_data.data.columns:
        all_parameters_list.append(col_name) if col_name not in all_parameters_list else all_parameters_list



In [42]:
parameters_widget = widgets.SelectMultiple(options=all_parameters_list, rows=20, description='All Parameters', style=style, 
                                          layout=widgets.Layout(width='75%'), disabled=False)
parameters_widget

SelectMultiple(description='All Parameters', layout=Layout(width='75%'), options=('HEX-PL1-1 shot #', 'HEX-PL1…

In [43]:
parameter_list_from_widget = list(parameters_widget.value)
# parameter_list_manual = ['Cell Length (mm)', 'Compressor Position (um)', 'Gas Jet On ?', 'HTT-WFS-400nm himg exists', 
#                          'HTT-WFS-400nm png file exists', 'HTT-WFS-800nm himg exists', 'HTT-WFS-800nm png file exists', 
#                          'Lamp Timing (us)', 'Pressure Measured (mbar)', 'Pressure Measured (torr)', 'Pressure Set (mbar)', 
#                          'Pressure Set (torr)', 'Probe Delay (ns)']
parameter_list_manual = [ 'Gas Jet On ?', 'Drive Pulse Delay (ns)', 'Gas Jet Backing Pressure (psi)','Compression', 
                         'CAM-2BL-F3M5 mean counts T', 'HPD_Energy', 'HPDmodeMaxCounts',
                         '1BL Delay', 'Amp2Attenuator', 'Amplifier 1 [mJ]', 'Amplifier 2 [J]', 
                         'Amplifier 3 [J]', 'Comp-2BL-separation [mm]', 
                          'HPD stage position','HPD2 stage position',  
                         'Input Delta R (um)', 'Input Delta X (um)', 'Input Delta Y (um)', 'Jet Length (cm)', 
                         ]


parameter_list = parameter_list_from_widget + parameter_list_manual
print(parameter_list)

['Gas Jet On ?', 'Drive Pulse Delay (ns)', 'Gas Jet Backing Pressure (psi)', 'Compression', 'CAM-2BL-F3M5 mean counts T', 'HPD_Energy', 'HPDmodeMaxCounts', '1BL Delay', 'Amp2Attenuator', 'Amplifier 1 [mJ]', 'Amplifier 2 [J]', 'Amplifier 3 [J]', 'Comp-2BL-separation [mm]', 'HPD stage position', 'HPD2 stage position', 'Input Delta R (um)', 'Input Delta X (um)', 'Input Delta Y (um)', 'Jet Length (cm)']


In [44]:
columns = ['year', 'month', 'day', 'scan', '# shots', 'scan_parameter', 'scan_info_txt']
for parameter in parameter_list:
    columns.append('%s MEAN' %parameter)
#     columns.append('%s STD' %parameter)
columns.append('sfilename')

scan_sheet = pd.DataFrame(columns=columns)

In [45]:
for i in range(len(sfilename_list)):
    sfilename = sfilename_list[i]
    scan_data = ScanDataAnalyzer(sfilename=sfilename)
    
    scan_data.load_scan_data(
        # search_replace_filename=search_replace_filename, 
        # column_math_filename=column_math_filename,
    )
    
    if not scan_data.aborted:
        scan_info_text = scan_data.get_scan_info_text()
    
        scan_sheet.loc[i, 'year'] = scan_data.year
        scan_sheet.loc[i, 'month'] = scan_data.month
        scan_sheet.loc[i, 'day'] = scan_data.day
        scan_sheet.loc[i, 'scan'] = scan_data.scan
        scan_sheet.loc[i, '# shots'] = len(scan_data.data)
        scan_sheet.loc[i, 'scan_parameter'] = scan_data.scan_parameter
        scan_sheet.loc[i, 'scan_info_txt'] = scan_info_text
        scan_sheet.loc[i, 'sfilename'] = sfilename
    
    for parameter in parameter_list:
        if parameter in scan_data.data.columns:
            scan_sheet.loc[i, '%s MEAN' %parameter] = np.nanmean(scan_data.data[parameter]) 
                

In [46]:
scan_sheet

,year,month,day,scan,# shots,scan_parameter,scan_info_txt,Gas Jet On ? MEAN,Drive Pulse Delay (ns) MEAN,Gas Jet Backing Pressure (psi) MEAN,...,Amplifier 2 [J] MEAN,Amplifier 3 [J] MEAN,Comp-2BL-separation [mm] MEAN,HPD stage position MEAN,HPD2 stage position MEAN,Input Delta R (um) MEAN,Input Delta X (um) MEAN,Input Delta Y (um) MEAN,Jet Length (cm) MEAN,sfilename
0,2025,7,28,1.0,100,HPD stage position,SMF hpd scan PL! HPD in vacuum,NaN,NaN,NaN,...,0.025805,0.09875,NaN,19.999985,NaN,NaN,NaN,NaN,NaN,N:/data/Y2025\07-Jul\25_0728\analysis\s1.txt
1,2025,7,28,2,100,HPD stage position,smf astig correction HPD ESP1 y moved by -0.3,NaN,NaN,NaN,...,0.027562,0.101222,NaN,19.999985,NaN,NaN,NaN,NaN,NaN,N:/data/Y2025\07-Jul\25_0728\analysis\s2.txt
2,2025,7,28,3.0,65,HPD stage position,"repeat last scan , did not save",NaN,NaN,NaN,...,0.02228,0.096429,NaN,16.999965,NaN,NaN,NaN,NaN,NaN,N:/data/Y2025\07-Jul\25_0728\analysis\s3.txt
3,2025,7,28,4,6,HPD stage position,moved esp 1 y by -0.05 more,NaN,NaN,NaN,...,0.035279,0.10972,NaN,5.333323,NaN,NaN,NaN,NaN,NaN,N:/data/Y2025\07-Jul\25_0728\analysis\s4.txt
4,2025,7,28,5.0,65,HPD stage position,repeat last scan,NaN,NaN,NaN,...,0.018913,0.102971,NaN,16.999965,NaN,NaN,NaN,NaN,NaN,N:/data/Y2025\07-Jul\25_0728\analysis\s5.txt
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,2025,8,14,9.0,130,Compression,compressor scan with 12.8mm glass inserted,NaN,NaN,NaN,...,18.647826,14.846723,NaN,0.044831,NaN,NaN,NaN,NaN,NaN,N:/data/Y2025\08-Aug\25_0814\analysis\s9.txt
86,2025,8,14,10.0,250,Shotnumber,"no scan to find correlation , LB FROG has a 12...",NaN,NaN,NaN,...,18.723878,14.909653,NaN,0.044831,NaN,NaN,NaN,NaN,NaN,N:/data/Y2025\08-Aug\25_0814\analysis\s10.txt
87,2025,8,14,11.0,130,Compression,compressor scan with LB FORG inserting a 5mm g...,NaN,NaN,NaN,...,18.726418,14.865655,NaN,0.044831,NaN,NaN,NaN,NaN,NaN,N:/data/Y2025\08-Aug\25_0814\analysis\s11.txt
88,2025,8,14,12.0,500,Shotnumber,no scan for 500 shots,NaN,NaN,NaN,...,18.704897,14.85557,NaN,0.044831,NaN,NaN,NaN,NaN,NaN,N:/data/Y2025\08-Aug\25_0814\analysis\s12.txt
